In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

In [ ]:
# Load just the blight survey data
blight_filepath = "../../data/blight_survey_data/20250527_DLBA_survey_data_UM_Detroit.xlsx"
blight_df = pd.read_excel(blight_filepath)
blight_df.columns

In [ ]:
blight_df.head()

In [ ]:
blight_df.info()

In [ ]:
blight_df.describe()

In [ ]:
blight_df.isnull().sum()

In [ ]:
blight_df['FINAL_DETERMINATION'].value_counts()

In [ ]:
blight_df['FINAL_DETERMINATION_2'].value_counts()

In [ ]:
blight_df['FIELD_DETERMINATION'].value_counts()

### Questions for blight data

1. What are the classes in blight, dependent on the above "FINAL_DETERMINATION"?
- Looks like "Other resolution pathways (salvage)" is highest blight?
- Looks like "No action" and "Vacant" are lowest blight?
- Looks like "Noticeable evidence of blight" of 26 labels, is 1?

1a. How does urbanworm do the label management here?

1b. What's the diff between FINAL_DETERMINATION, FINAL_DETERMINATION_2, and FIELD_DETERMINATION change?

2. What are the important features in predicting blight?
- Let's unpack all of the columns, seems like we could just pick 1 to start. ROOF_CONDITION?

In [ ]:
blight_df.columns

In [ ]:
blight_tight_df = blight_df[['PARCEL_ID', 'DATE_SURVEYED', 'IS_OCCUPIED', 'FIRE_DAMAGE_CONDITION', 'ROOF_CONDITION', 'OPENINGS_CONDITION', 'IS_OPEN_TO_TRESPASS', 'FIELD_DETERMINATION']]

In [ ]:
for col in blight_tight_df.columns:
    if col not in ['PARCEL_ID', 'DATE_SURVEYED']:
        print(blight_tight_df[col].value_counts())

In [ ]:
def make_labels(blight_df):
    """
    Create blight labels from FIELD_DETERMINATION column.
    
    Labels:
    0 = No/minimal blight (Salvage categories, Vacant not blighted)
    1 = Noticeable evidence of blight
    2 = Significant evidence of blight  
    3 = Extreme evidence of blight
    
    Excludes: 'Property in Pipeline' and 'ODM (Demo)'
    
    Returns: DataFrame with PARCEL_ID and BLIGHT_LABEL columns only
    """
    # Create label mapping
    label_mapping = {
        'No Action (Salvage)': 0,
        ' NAP (Salvage)': 0,
        'Other Resolution Pathways (Salvage)': 0,
        'Vacant (Not Blighted)': 0,
        'Noticeable Evidence of Blight': 1,
        'Significant Evidence of Blight': 2,
        'Extreme Evidence of Blight': 3
    }
    
    # Filter out unwanted categories
    exclude_categories = ['Property in Pipeline', 'ODM (Demo)']
    filtered_df = blight_df[~blight_df['FIELD_DETERMINATION'].isin(exclude_categories)].copy()
    
    # Create blight_labels dataframe
    blight_labels = filtered_df[['PARCEL_ID', 'FIELD_DETERMINATION']].copy()
    blight_labels['BLIGHT_LABEL'] = blight_labels['FIELD_DETERMINATION'].map(label_mapping)
    blight_labels = blight_labels[['PARCEL_ID', 'BLIGHT_LABEL']]
    
    # Remove any rows with missing labels
    blight_labels = blight_labels.dropna()
    
    # Convert to integer type
    blight_labels['BLIGHT_LABEL'] = blight_labels['BLIGHT_LABEL'].astype(int)
    
    return blight_labels

In [ ]:
blight_labels_df = make_labels(blight_df)

In [ ]:
blight_labels_df['BLIGHT_LABEL'].value_counts()

In [ ]:
def make_features(blight_df):
    """
    Create features dataframe for training.
    
    Includes: PARCEL_ID, DATE_SURVEYED, IS_OCCUPIED, FIRE_DAMAGE_CONDITION, 
              ROOF_CONDITION, OPENINGS_CONDITION, IS_OPEN_TO_TRESPASS
    
    Returns: DataFrame with selected feature columns
    """
    feature_columns = [
        'PARCEL_ID', 
        'DATE_SURVEYED', 
        'IS_OCCUPIED', 
        'FIRE_DAMAGE_CONDITION', 
        'ROOF_CONDITION', 
        'OPENINGS_CONDITION', 
        'IS_OPEN_TO_TRESPASS'
    ]
    
    # Create features dataframe
    blight_features_df = blight_df[feature_columns].copy()
    
    return blight_features_df

In [ ]:
blight_features_df = make_features(blight_df)

In [ ]:
# Let's save off the datasets so we can train a model...
blight_labels_df.to_csv("../training_data/blight_labels.csv")
blight_features_df.to_csv("../training_data/blight_features.csv")